In [ ]:
import pickle
import ml.structures
import ml.dataset.corpus as corpus
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from keras.models import load_model

from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import RMSprop


from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.engine.topology import Input

from sklearn.metrics import accuracy_score
from keras.layers import concatenate, Merge
from keras.layers.merge import Concatenate

import keras
from keras.utils import to_categorical

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [ ]:
def make_16(chord):
    if (chord.duration > 0):
        c = 16 / (128 / chord.duration)
    else:
        return 0, [0]
    pre_array = np.array([note.number % 12 for note in chord.notes])
    array = set()
    for el in pre_array:
        array.add(el)
        if len(array) == 4:
            break
    array = np.array(list(array))
    array = np.sort(array)
    if len(array) == 0:
        array = [0]
    if c >= 1:
        return 1, np.tile(array, (int(c), 1))
    else:
        return 0, [0]

In [ ]:
def cut_song(song, notes, chords):
    chords_track = []
    notes_track = np.array([])
    flag = 0
    for note in song.tracks[0].chords:
        if (make_16(note)[0] == 0):
            flag = 1
        notes_track = np.append(notes_track, make_16(note)[1])
    for chord in song.tracks[1].chords:
        if (make_16(chord)[0] == 0):
            flag = 1
        chords_track.append(make_16(chord)[1])
    if flag == 1:
        return 0, notes, chords
    return 1, notes_track, chords_track

In [ ]:
def norm_array(X, size):
    for j in range(len(X)):
        X[j] = np.unique(np.array(X[j]))[:size]
        array = np.zeros(size-len(X[j])) - 1
        X[j] = np.append(X[j], array)
    X = np.array(X)
    return X

In [ ]:
def create_X_Y(chords):
    real_chords = []
    y = []
    for i, subchord in enumerate(chords):
        if i != len(chords) - 1:
            for ssubchord in subchord:
                real_chords.append(ssubchord)
            size = len(subchord)
            for j in range(size):
                y.append(chords[i+1][-1])
    return real_chords, y

In [ ]:
def notes_to_np(notes):
    for i in range(len(notes)):
        notes[i] = np.array(notes[i])
    notes = np.array(notes)

In [ ]:
def create_unique_chords(X, unique_chords):
    for x in X:
        unique_chords = np.append(unique_chords, np.array(x))
    return unique_chords

In [ ]:
def create_coded(X):
    return np.zeros(len(X))


In [ ]:
def encode(X, coded_X):
    for j, k in enumerate(X):
        for i, unique_chord in enumerate(unique_chords):
            if np.array_equal(k, unique_chord):
                coded_X[j] = i

In [ ]:
model_chords = Sequential()
model_notes = Sequential()

model_chords.add(Embedding(794, 20, input_length=1, batch_input_shape=(1,1)))
model_notes.add(Embedding(128, 20, input_length=1, batch_input_shape=(1,1)))

merged_model = Sequential()
merged = Merge([model_chords, model_notes])
merged_model.add(merged)
merged_model.add(LSTM(20, stateful=True))
merged_model.add(Dense(794, activation='softmax'))
merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

y_real = []
y_pred = []
unique_chords = np.zeros(4) - 1

with open('gtp_dataset_final.pickle', 'rb') as input_file:
    i = 0
    while True:
        try:
            i += 1
            song = ml.structures.Song()
            song.undump(input_file)
            chords = []
            notes = []
            flag_notes_chords = cut_song(song, notes, chords)
            if (flag_notes_chords[0] == 0) :
                continue
            notes = flag_notes_chords[1]
            chords = flag_notes_chords[2]
            X = []
            Y = []
            X, Y = create_X_Y(chords)
            X = norm_array(X, 4)
            Y = norm_array(Y, 4)
            notes_to_np(notes)
            unique_chords = create_unique_chords(X, unique_chords)
            unique_chords = create_unique_chords(Y, unique_chords)
            unique_chords = unique_chords.reshape((-1, 4))
            unique_chords = np.unique(unique_chords, axis=0)
            unique_chords = list(unique_chords)
            coded_X = []
            coded_Y = []
            coded_X = create_coded(X)
            coded_Y = create_coded(Y)
            encode(X, coded_X)
            encode(Y, coded_Y)
            X = np.array(coded_X)
            y = np.array(coded_Y)
            X_notes = notes
            X_notes = X_notes[:len(X)]
            X = X[:len(X_notes)]
            y = y[:len(X)]
            if (i < 27100):
                merged_model.fit([X, X_notes], y, epochs=1, batch_size=1, verbose=1)
                merged_model.reset_states()
            else:
                y_pred.append(merged_model.predict([X, X_notes], batch_size=1, verbose=1).argmax(axis=1))
                y_real.append(y)
            
            
        except EOFError:
            break

In [ ]:
df = pd.DataFrame(unique_chords)
df.to_csv("vocab.csv")

In [ ]:
x = pd.read_csv("vocab.csv")
x = x.values.T[1:].T

In [ ]:
x

In [ ]:
merged_model.save("NN_model.h5")